# AIDA Nanopublication Generator (Corrected)

Creates AIDA (Atomic, Independent, Declarative, Absolute) sentence nanopublications from a JSON configuration file.

**Template:** [AIDA Sentence Template](https://w3id.org/np/RALmXhDw3rHcMveTgbv8VtWxijUHwnSqhCmtJFIPKWVaA)

## AIDA Sentences
- **A**tomic: A sentence describing one thought that cannot be further broken down
- **I**ndependent: A sentence that can stand on its own, without external references
- **D**eclarative: A complete sentence ending with a full stop that could be true or false
- **A**bsolute: A sentence describing the core of a claim ignoring uncertainty

---

## Changes from Original
- Uses `rdflib.Dataset` instead of string concatenation
- Uses temporary namespace `http://purl.org/nanopub/temp/np/` (replaced on signing)
- Proper signing workflow with nanopub library

---
# 📝 SECTION 1: INPUT FILE (EDIT THIS)
---

In [13]:
# Path to your AIDA JSON config file
CONFIG_FILE = "../config/vbae208/vbae208_aida.json"
CONFIG_FILE = "../config/clenet2025/clenet2025_aida.json"
OUTPUT_DIR = "../output/aida"

---
# ⚙️ SECTION 2: SETUP
---

In [14]:
# Install dependencies (uncomment if needed)
# !pip install nanopub rdflib

In [15]:
import json
import urllib.parse
from pathlib import Path
from datetime import datetime, timezone
from rdflib import Dataset, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD, FOAF

# Namespaces (matching Nanodash)
NP = Namespace("http://www.nanopub.org/nschema#")
DCT = Namespace("http://purl.org/dc/terms/")
NT = Namespace("https://w3id.org/np/o/ntemplate/")
NPX = Namespace("http://purl.org/nanopub/x/")
PROV = Namespace("http://www.w3.org/ns/prov#")
ORCID = Namespace("https://orcid.org/")
HYCL = Namespace("http://purl.org/petapico/o/hycl#")
CITO = Namespace("http://purl.org/spar/cito/")
SCHEMA = Namespace("http://schema.org/")
SKOS = Namespace("http://www.w3.org/2004/02/skos/core#")
AIDA = Namespace("http://purl.org/aida/")

# Template URIs
AIDA_TEMPLATE = URIRef("https://w3id.org/np/RALmXhDw3rHcMveTgbv8VtWxijUHwnSqhCmtJFIPKWVaA")
PROV_TEMPLATE = URIRef("https://w3id.org/np/RA7lSq6MuK_TIC6JMSHvLtee3lpLoZDOqLJCLXevnrPoU")
PUBINFO_TEMPLATE_1 = URIRef("https://w3id.org/np/RA0J4vUn_dekg-U1kK3AOEt02p9mT2WO03uGxLDec1jLw")
PUBINFO_TEMPLATE_2 = URIRef("https://w3id.org/np/RAukAcWHRDlkqxk7H2XNSegc1WnHI569INvNr-xdptDGI")

# AIDA-specific constants
HYCL_AIDA_SENTENCE = URIRef("http://purl.org/petapico/o/hycl#AIDA-Sentence")
HYCL_NS = URIRef("http://purl.org/petapico/o/hycl")

print("✓ Setup complete")

✓ Setup complete


---
# 📖 SECTION 3: LOAD & VALIDATE
---

In [16]:
# Load configuration
print(f"Loading: {CONFIG_FILE}")

with open(CONFIG_FILE, 'r', encoding='utf-8') as f:
    config = json.load(f)

# Extract metadata
metadata = config.get('metadata', {})
AUTHOR_ORCID = metadata.get('creator_orcid')
AUTHOR_NAME = metadata.get('creator_name')
IS_PART_OF = metadata.get('is_part_of', {})

# Validate
errors = []
if not AUTHOR_ORCID:
    errors.append("metadata.creator_orcid is required")
if not AUTHOR_NAME:
    errors.append("metadata.creator_name is required")
if not config.get('nanopublications'):
    errors.append("nanopublications list is required")

if errors:
    print("❌ Validation errors:")
    for e in errors:
        print(f"   - {e}")
    raise ValueError("Please fix the errors in your JSON file")

print(f"✓ Loaded {len(config['nanopublications'])} AIDA nanopubs to generate")
print(f"✓ Author: {AUTHOR_NAME} ({AUTHOR_ORCID})")
if IS_PART_OF:
    print(f"✓ Part of: {IS_PART_OF.get('label', IS_PART_OF.get('uri'))}")

Loading: ../config/clenet2025/clenet2025_aida.json
✓ Loaded 6 AIDA nanopubs to generate
✓ Author: Anne Fouilloux (0000-0002-1784-2920)
✓ Part of: Quantum Computing Applications for Biodiversity Research - Systematic Review


---
# 🔨 SECTION 4: BUILD NANOPUBLICATIONS
---

In [17]:
def create_aida_nanopub(np_config, metadata):
    """
    Create an AIDA sentence nanopublication using rdflib Dataset.
    
    Args:
        np_config: dict with AIDA sentence configuration
        metadata: dict with author and part_of info
    
    Returns:
        tuple: (Dataset, label)
    """
    # CRITICAL: Use temporary namespace that gets replaced when signing
    TEMP_NP = Namespace("http://purl.org/nanopub/temp/np/")
    
    this_np = URIRef("http://purl.org/nanopub/temp/np/")
    head_graph = URIRef("http://purl.org/nanopub/temp/np/Head")
    assertion_graph = URIRef("http://purl.org/nanopub/temp/np/assertion")
    provenance_graph = URIRef("http://purl.org/nanopub/temp/np/provenance")
    pubinfo_graph = URIRef("http://purl.org/nanopub/temp/np/pubinfo")
    
    author_uri = ORCID[metadata['creator_orcid']]
    
    # Create AIDA sentence URI (URL-encoded)
    aida_sentence = np_config['aida_sentence']
    aida_uri = URIRef(f"http://purl.org/aida/{urllib.parse.quote(aida_sentence, safe='')}")
    
    # Create Dataset
    ds = Dataset()
    
    # Bind prefixes
    ds.bind("this", "http://purl.org/nanopub/temp/np/")
    ds.bind("sub", TEMP_NP)
    ds.bind("np", NP)
    ds.bind("dct", DCT)
    ds.bind("nt", NT)
    ds.bind("npx", NPX)
    ds.bind("xsd", XSD)
    ds.bind("rdfs", RDFS)
    ds.bind("orcid", ORCID)
    ds.bind("prov", PROV)
    ds.bind("foaf", FOAF)
    ds.bind("hycl", HYCL)
    ds.bind("cito", CITO)
    ds.bind("schema", SCHEMA)
    ds.bind("skos", SKOS)
    
    # HEAD graph
    head = ds.graph(head_graph)
    head.add((this_np, RDF.type, NP.Nanopublication))
    head.add((this_np, NP.hasAssertion, assertion_graph))
    head.add((this_np, NP.hasProvenance, provenance_graph))
    head.add((this_np, NP.hasPublicationInfo, pubinfo_graph))
    
    # ASSERTION graph
    assertion = ds.graph(assertion_graph)
    assertion.add((aida_uri, RDF.type, HYCL_AIDA_SENTENCE))
    
    # Related publication
    if np_config.get('related_publication'):
        pub = np_config['related_publication']
        pub_uri = URIRef(pub if pub.startswith('http') else f"https://doi.org/{pub}")
        assertion.add((aida_uri, CITO.obtainsSupportFrom, pub_uri))
    
    # Related dataset
    if np_config.get('related_dataset'):
        assertion.add((aida_uri, CITO.obtainsSupportFrom, URIRef(np_config['related_dataset'])))
    
    # Topics
    topics = np_config.get('topics', [])
    for topic in topics:
        if topic.get('uri'):
            assertion.add((aida_uri, SCHEMA.about, URIRef(topic['uri'])))
    
    # Related project
    if np_config.get('related_project'):
        assertion.add((aida_uri, SKOS.related, URIRef(np_config['related_project'])))
    
    # isPartOf (in assertion for AIDA)
    is_part_of = metadata.get('is_part_of', {})
    if is_part_of.get('uri'):
        assertion.add((aida_uri, DCT.isPartOf, URIRef(is_part_of['uri'])))
    
    # PROVENANCE graph
    provenance = ds.graph(provenance_graph)
    provenance.add((assertion_graph, PROV.wasAttributedTo, author_uri))
    
    # PUBINFO graph
    pubinfo = ds.graph(pubinfo_graph)
    pubinfo.add((author_uri, FOAF.name, Literal(metadata['creator_name'])))
    
    now = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%S.000Z")
    pubinfo.add((this_np, DCT.created, Literal(now, datatype=XSD.dateTime)))
    pubinfo.add((this_np, DCT.creator, author_uri))
    pubinfo.add((this_np, DCT.license, URIRef("https://creativecommons.org/licenses/by/4.0/")))
    pubinfo.add((this_np, NPX.wasCreatedAt, URIRef("https://sciencelive4all.org/")))
    
    # Nanopub types
    pubinfo.add((this_np, NPX.hasNanopubType, HYCL_AIDA_SENTENCE))
    pubinfo.add((this_np, NPX.hasNanopubType, HYCL_NS))
    
    # Introduces
    pubinfo.add((this_np, NPX.introduces, aida_uri))
    
    # Label
    label_text = aida_sentence[:100] + "..." if len(aida_sentence) > 100 else aida_sentence
    label = f"AIDA sentence: {label_text}"
    pubinfo.add((this_np, RDFS.label, Literal(label)))
    
    # Template references
    pubinfo.add((this_np, NT.wasCreatedFromTemplate, AIDA_TEMPLATE))
    pubinfo.add((this_np, NT.wasCreatedFromProvenanceTemplate, PROV_TEMPLATE))
    pubinfo.add((this_np, NT.wasCreatedFromPubinfoTemplate, PUBINFO_TEMPLATE_1))
    pubinfo.add((this_np, NT.wasCreatedFromPubinfoTemplate, PUBINFO_TEMPLATE_2))
    
    # Add topic labels in pubinfo (for Nanodash display)
    for topic in topics:
        if topic.get('uri') and topic.get('label'):
            pubinfo.add((URIRef(topic['uri']), NT.hasLabelFromApi, Literal(topic['label'])))
    
    return ds, label

print("✓ Function defined")

✓ Function defined


In [18]:
# Create output directory
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

# Generate all nanopublications
generated_files = []

for np_config in config['nanopublications']:
    # Create nanopub
    ds, label = create_aida_nanopub(np_config, metadata)
    
    # Save to file
    output_file = Path(OUTPUT_DIR) / f"{np_config['id']}.trig"
    ds.serialize(destination=str(output_file), format='trig')
    generated_files.append(output_file)
    
    print(f"✓ Generated: {output_file}")

print(f"\nTotal generated: {len(generated_files)} nanopublications")

✓ Generated: ../output/aida/aida_clenet_01.trig
✓ Generated: ../output/aida/aida_clenet_02.trig
✓ Generated: ../output/aida/aida_clenet_03.trig
✓ Generated: ../output/aida/aida_clenet_04.trig
✓ Generated: ../output/aida/aida_clenet_05.trig
✓ Generated: ../output/aida/aida_clenet_06.trig

Total generated: 6 nanopublications


---
# 📄 SECTION 5: PREVIEW & OUTPUT
---

In [19]:
# Preview first generated nanopublication
if generated_files:
    print(f"Preview of {generated_files[0]}:\n")
    print("=" * 80)
    with open(generated_files[0], 'r') as f:
        print(f.read())

Preview of ../output/aida/aida_clenet_01.trig:

@prefix cito: <http://purl.org/spar/cito/> .
@prefix dct: <http://purl.org/dc/terms/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix hycl: <http://purl.org/petapico/o/hycl#> .
@prefix np: <http://www.nanopub.org/nschema#> .
@prefix npx: <http://purl.org/nanopub/x/> .
@prefix nt: <https://w3id.org/np/o/ntemplate/> .
@prefix orcid: <https://orcid.org/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema1: <http://schema.org/> .
@prefix sub: <http://purl.org/nanopub/temp/np/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

sub:Head {
    sub: a np:Nanopublication ;
        np:hasAssertion sub:assertion ;
        np:hasProvenance sub:provenance ;
        np:hasPublicationInfo sub:pubinfo .
}

sub:pubinfo {
    <http://www.wikidata.org/entity/Q17995793> nt:hasLabelFromApi "quantum computing" .

    <http://www.wikidata.org/entity/Q232207> nt:hasLabelFromApi "Monte 

---
# 🚀 SECTION 6: SIGN & PUBLISH (OPTIONAL)
---

In [20]:
PUBLISH = True  # Set to True when ready to publish
USE_TEST_SERVER = False  # Set to False for production

In [21]:
if PUBLISH:
    from nanopub import Nanopub, NanopubConf, load_profile
    
    profile = load_profile()
    print(f"Loaded profile: {profile.name}")
    
    conf = NanopubConf(profile=profile, use_test_server=USE_TEST_SERVER)
    
    for trig_file in generated_files:
        np_obj = Nanopub(rdf=trig_file, conf=conf)
        np_obj.sign()
        
        signed_path = trig_file.with_suffix('.signed.trig')
        np_obj.store(signed_path)
        print(f"✓ Signed: {signed_path}")
        
        np_obj.publish()
        print(f"✓ Published: {np_obj.source_uri}")
else:
    print("Publishing disabled. Set PUBLISH = True to enable.")
    print("\nManual signing:")
    for f in generated_files:
        print(f"  nanopub sign {f}")
        print(f"  nanopub publish {f.stem}.signed.trig")

Loaded profile: Anne Fouilloux
✓ Signed: ../output/aida/aida_clenet_01.signed.trig
✓ Published: https://w3id.org/np/RAZ4to5WjeFGrCaM8S_i5Lt64BSl33HJbBorGBHd5QFdw
✓ Signed: ../output/aida/aida_clenet_02.signed.trig
✓ Published: https://w3id.org/np/RAyS9cLWE4ugfPEo0eqU-nYVAgQX7k--ix8DIIJtoQyPk
✓ Signed: ../output/aida/aida_clenet_03.signed.trig
✓ Published: https://w3id.org/np/RAHy0Q6a58PaBFAljetALzmE57Xhq2ZyYccSFT1Vqk-kI
✓ Signed: ../output/aida/aida_clenet_04.signed.trig
✓ Published: https://w3id.org/np/RAtf9ptkM0NH0VDn_8xj-H80Tmz2xA-cjvLUvAYdzhWiA
✓ Signed: ../output/aida/aida_clenet_05.signed.trig
✓ Published: https://w3id.org/np/RA9sW6IvKmSZTXV5BIWTXlrKdn4ndHgUmsfQ3A3YMY0mw
✓ Signed: ../output/aida/aida_clenet_06.signed.trig
✓ Published: https://w3id.org/np/RA-kvfgs3--v7HiNBZN0s7YVpMAv8v0aE2YoA-io12UGo


---
# 📋 JSON TEMPLATE

```json
{
  "metadata": {
    "source_paper": { "title": "...", "doi": "..." },
    "creator_orcid": "0000-0000-0000-0000",
    "creator_name": "Your Name",
    "is_part_of": {
      "uri": "https://w3id.org/np/YOUR-REVIEW-URI",
      "label": "Your Systematic Review Title"
    }
  },
  "nanopublications": [
    {
      "id": "aida_example_01",
      "aida_sentence": "Your AIDA sentence here.",
      "related_publication": "10.xxxx/xxxxx",
      "topics": [
        {"uri": "http://www.wikidata.org/entity/Qxxxxx", "label": "topic name"}
      ]
    }
  ]
}
```
---